In [2]:
import pickle
import re
from sklearn.metrics import confusion_matrix
import requests, bs4
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
from sklearn import svm
from sklearn.linear_model import LogisticRegression
from sklearn.svm import LinearSVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.model_selection import cross_val_score
from sklearn import preprocessing
from sklearn.feature_selection import RFE
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
import os

In [17]:
db = pd.read_csv('2017_2019stats.csv')
moneydb = pd.read_csv('MLOdds.csv')
spreaddb = pd.read_csv('Spreads.csv')

In [4]:
team_abbrevs = {
    'Cleveland': 'CLE',
    'Boston':'BOS',
    'Washington':'WAS',
    'Charlotte':'CHO',
    'Minnesota':'MIN',
    'Dallas':'DAL',
    'Milwaukee':'MIL',
    'Philadelphia':'PHI',
    'Phoenix':'PHO',
    'L.A. Lakers':'LAL',
    'Utah':'UTA',
    'Sacramento':'SAC',
    'New York':'NYK',
    'New Orleans':'NOP',
    'Detroit':'DET',
    'Atlanta':'ATL',
    'Chicago':'CHI',
    'Miami':'MIA',
    'Memphis':'MEM',
    'Golden State':'GSW',
    'Denver':'DEN',
    'Brooklyn':'BRK',
    'L.A. Clippers':'LAC',
    'Portland':'POR',
    'Indiana':'IND',
    'San Antonio':'SAS',
    'Houston':'HOU',
    'Oklahoma City':'OKC',
    'Toronto':'TOR',
    'Orlando':'ORL'
}

In [283]:
# Moneyline scrape
scraped_dates = []
moneydb = pd.DataFrame()

for date in db['Date']:
    
    if date not in scraped_dates:
        print("Date: ", date)
        url = f'https://www.sportsbookreview.com/betting-odds/nba-basketball/money-line/?date={date}'
        res = requests.get(url, 'html.parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'lxml')
        linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
        test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
        teams = [team_abbrevs[team.get_text()] for team in test_teams]
        test_lines = linesoup.find_all('span', {'class':'opener'})
        lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') and len(line.get_text()) > 1 or line.get_text().startswith('+') and len(line.get_text()) > 1]
        print("teams:",len(teams),"lines",len(lines))
        moneylines = {'Team':teams, 'ML Odds': lines, 'Date':[date]*len(teams)}
        MLdb = pd.DataFrame.from_dict(moneylines)
        moneydb = pd.concat([moneydb, MLdb])
        scraped_dates.append(date)
        
# moneydb.to_csv('MLodds.csv')

Date:  20181017
teams: 22 lines 22
Date:  20181019
teams: 18 lines 18
Date:  20181021
teams: 8 lines 8
Date:  20181024
teams: 22 lines 22
Date:  20181025
teams: 8 lines 8
Date:  20181027
teams: 18 lines 18
Date:  20181030
teams: 16 lines 16
Date:  20181101
teams: 12 lines 12
Date:  20181103
teams: 16 lines 16
Date:  20181105
teams: 18 lines 18
Date:  20181107
teams: 20 lines 20
Date:  20181110
teams: 20 lines 20
Date:  20181113
teams: 6 lines 6
Date:  20181114
teams: 22 lines 22
Date:  20181119
teams: 18 lines 18
Date:  20181121
teams: 26 lines 26
Date:  20181123
teams: 28 lines 28
Date:  20181124
teams: 14 lines 14
Date:  20181126
teams: 14 lines 14
Date:  20181128
teams: 20 lines 20
Date:  20181130
teams: 22 lines 22
Date:  20181201
teams: 14 lines 14
Date:  20181203
teams: 14 lines 14
Date:  20181205
teams: 20 lines 20
Date:  20181207
teams: 20 lines 20
Date:  20181208
teams: 18 lines 18
Date:  20181210
teams: 22 lines 22
Date:  20181212
teams: 22 lines 22
Date:  20181214
teams: 18 

teams: 22 lines 22
Date:  20180319
teams: 16 lines 16
Date:  20180321
teams: 16 lines 16
Date:  20180323
teams: 20 lines 20
Date:  20180325
teams: 18 lines 18
Date:  20180327
teams: 16 lines 16
Date:  20180328
teams: 16 lines 16
Date:  20180330
teams: 18 lines 18
Date:  20180401
teams: 26 lines 26
Date:  20180403
teams: 26 lines 26
Date:  20180405
teams: 12 lines 12
Date:  20180406
teams: 20 lines 20
Date:  20180409
teams: 18 lines 18
Date:  20180411
teams: 24 lines 24
Date:  20171018
teams: 22 lines 22
Date:  20171026
teams: 10 lines 10
Date:  20171030
teams: 18 lines 18
Date:  20171106
teams: 6 lines 6
Date:  20171108
teams: 10 lines 10
Date:  20171110
teams: 16 lines 16
Date:  20171112
teams: 8 lines 8
Date:  20171114
teams: 6 lines 6
Date:  20171116
teams: 4 lines 4
Date:  20171118
teams: 14 lines 14
Date:  20171125
teams: 20 lines 20
Date:  20171210
teams: 12 lines 12
Date:  20171211
teams: 12 lines 12
Date:  20171213
teams: 18 lines 18
Date:  20171215
teams: 22 lines 22
Date:  20

teams: 6 lines 6
Date:  20170313
teams: 16 lines 16
Date:  20170328
teams: 16 lines 16
Date:  20170105
teams: 16 lines 16
Date:  20170120
teams: 18 lines 18
Date:  20170207
teams: 6 lines 6
Date:  20170302
teams: 6 lines 6
Date:  20170411
teams: 10 lines 10
Date:  20170101
teams: 10 lines 10
Date:  20170109
teams: 6 lines 6
Date:  20170115
teams: 14 lines 14
Date:  20170117
teams: 10 lines 10
Date:  20170122
teams: 8 lines 8
Date:  20170126
teams: 8 lines 8
Date:  20170212
teams: 8 lines 8
Date:  20170321
teams: 16 lines 16
Date:  20170401
teams: 10 lines 10
Date:  20170403
teams: 2 lines 2
Date:  20170112
teams: 12 lines 12
Date:  20170323
teams: 10 lines 10
Date:  20161110
teams: 8 lines 8
Date:  20161215
teams: 10 lines 10
Date:  20161031
teams: 8 lines 8
Date:  20161120
teams: 12 lines 12
Date:  20161122
teams: 8 lines 8
Date:  20161204
teams: 8 lines 8


In [284]:
moneydb.to_csv('MLodds.csv', index = False)

In [289]:
# Point spread scrape
scraped_dates = []
spreaddb = pd.DataFrame()

for date in db['Date']:
    if date not in scraped_dates:
        print("Date: ", date)
        url = f'https://www.sportsbookreview.com/betting-odds/nba-basketball/pointspread/?date={date}'
        res = requests.get(url, 'html.parser')
        res.raise_for_status()
        soup = bs4.BeautifulSoup(res.text, 'lxml')
        linesoup = soup.find('div', {'id':'bettingOddsGridContainer'})
        test_teams = linesoup.find_all('span', {'class':'_3O1Gx'})
        teams = [team_abbrevs[team.get_text()] for team in test_teams]
        test_lines = linesoup.find_all('span', {'class':'_3Nv_7 opener'})
        lines = [line.get_text() for line in test_lines if line.get_text().startswith('-') or line.get_text().startswith('+') or line.get_text().startswith('P')]
        print("teams:",len(teams),"lines",len(lines))
        spread = {'Team':teams, 'Spread': lines, 'Date':[date]*len(lines)}
        spreads = pd.DataFrame.from_dict(spread)
        spreaddb = pd.concat([spreaddb, spreads])
        scraped_dates.append(date)

spreaddb.to_csv('Spreads.csv', index = False)

Date:  20181017
teams: 22 lines 22
Date:  20181019
teams: 18 lines 18
Date:  20181021
teams: 8 lines 8
Date:  20181024
teams: 22 lines 22
Date:  20181025
teams: 8 lines 8
Date:  20181027
teams: 18 lines 18
Date:  20181030
teams: 16 lines 16
Date:  20181101
teams: 12 lines 12
Date:  20181103
teams: 16 lines 16
Date:  20181105
teams: 18 lines 18
Date:  20181107
teams: 20 lines 20
Date:  20181110
teams: 20 lines 20
Date:  20181113
teams: 6 lines 6
Date:  20181114
teams: 22 lines 22
Date:  20181119
teams: 18 lines 18
Date:  20181121
teams: 26 lines 26
Date:  20181123
teams: 28 lines 28
Date:  20181124
teams: 14 lines 14
Date:  20181126
teams: 14 lines 14
Date:  20181128
teams: 20 lines 20
Date:  20181130
teams: 22 lines 22
Date:  20181201
teams: 14 lines 14
Date:  20181203
teams: 14 lines 14
Date:  20181205
teams: 20 lines 20
Date:  20181207
teams: 20 lines 20
Date:  20181208
teams: 18 lines 18
Date:  20181210
teams: 22 lines 22
Date:  20181212
teams: 22 lines 22
Date:  20181214
teams: 18 

teams: 22 lines 22
Date:  20180319
teams: 16 lines 16
Date:  20180321
teams: 16 lines 16
Date:  20180323
teams: 20 lines 20
Date:  20180325
teams: 18 lines 18
Date:  20180327
teams: 16 lines 16
Date:  20180328
teams: 16 lines 16
Date:  20180330
teams: 18 lines 18
Date:  20180401
teams: 26 lines 26
Date:  20180403
teams: 26 lines 26
Date:  20180405
teams: 12 lines 12
Date:  20180406
teams: 20 lines 20
Date:  20180409
teams: 18 lines 18
Date:  20180411
teams: 24 lines 24
Date:  20171018
teams: 22 lines 22
Date:  20171026
teams: 10 lines 10
Date:  20171030
teams: 18 lines 18
Date:  20171106
teams: 6 lines 6
Date:  20171108
teams: 10 lines 10
Date:  20171110
teams: 16 lines 16
Date:  20171112
teams: 8 lines 8
Date:  20171114
teams: 6 lines 6
Date:  20171116
teams: 4 lines 4
Date:  20171118
teams: 14 lines 14
Date:  20171125
teams: 20 lines 20
Date:  20171210
teams: 12 lines 12
Date:  20171211
teams: 12 lines 12
Date:  20171213
teams: 18 lines 18
Date:  20171215
teams: 22 lines 22
Date:  20

teams: 6 lines 6
Date:  20170313
teams: 16 lines 16
Date:  20170328
teams: 16 lines 16
Date:  20170105
teams: 16 lines 16
Date:  20170120
teams: 18 lines 18
Date:  20170207
teams: 6 lines 6
Date:  20170302
teams: 6 lines 6
Date:  20170411
teams: 10 lines 10
Date:  20170101
teams: 10 lines 10
Date:  20170109
teams: 6 lines 6
Date:  20170115
teams: 14 lines 14
Date:  20170117
teams: 10 lines 10
Date:  20170122
teams: 8 lines 8
Date:  20170126
teams: 8 lines 8
Date:  20170212
teams: 8 lines 8
Date:  20170321
teams: 16 lines 16
Date:  20170401
teams: 10 lines 10
Date:  20170403
teams: 2 lines 2
Date:  20170112
teams: 12 lines 12
Date:  20170323
teams: 10 lines 10
Date:  20161110
teams: 8 lines 8
Date:  20161215
teams: 10 lines 10
Date:  20161031
teams: 8 lines 8
Date:  20161120
teams: 12 lines 12
Date:  20161122
teams: 8 lines 8
Date:  20161204
teams: 8 lines 8


In [27]:
db['Odds'] = np.zeros(len(db['Team']))
db['Line'] = np.zeros(len(db['Team']))

In [297]:
date=db['Date'][0]
team = db['Team'][0]
print(date, team)

20181017 CLE


In [323]:
#db['Odds'][0] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][0])]['ML Odds']
#db['Line'][0] = spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][0])]['Spread']

moneydb.reset_index(drop = True, inplace = True)
spreaddb.reset_index(drop = True, inplace = True)



In [18]:
print(re.sub("½", ".5", spreaddb['Spread'][0]))

for i, spread in enumerate(spreaddb['Spread']):
    spreaddb['Spread'][i] = re.sub("½", ".5", spread)
    print(spreaddb['Spread'][i])

+7
+7
-7
-1.5
+1.5
-2
+2


C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


+4.5
-4.5
+3.5
-3.5
+11.5
-11.5
+7.5
-7.5
+2
-2
-7
+7
PK
PK
PK
PK
-2
+2
+2.5
-2.5
+11
-11
+3
-3
+8.5
-8.5
+7.5
-7.5
+3.5
-3.5
-2
+2
+1.5
-1.5
+6.5
-6.5
+9
-9
-3.5
+3.5
-3
+3
+3
-3
-1.5
+1.5
+8
-8
+8
-8
-3
+3
+2.5
-2.5
+2.5
-2.5
+4.5
-4.5
-2.5
+2.5
-3
+3
+10
-10
+7
-7
-3
+3
+1.5
-1.5
-4
+4
-3
+3
-1
+1
+6
-6
+5
-5
-6
+6
+1.5
-1.5
+9
-9
+1.5
-1.5
+9
-9
+4
-4
+4.5
-4.5
+4.5
-4.5
+9
-9
+6.5
-6.5
+2.5
-2.5
+2.5
-2.5
+2
-2
+1.5
-1.5
+4.5
-4.5
-6
+6
-2
+2
+3
-3
+3.5
-3.5
+7.5
-7.5
-1.5
+1.5
+9.5
-9.5
-5.5
+5.5
-8
+8
+3.5
-3.5
+4
-4
+3.5
-3.5
PK
PK
+4.5
-4.5
+3.5
-3.5
+2.5
-2.5
+4.5
-4.5
+2
-2
-1
+1
+14
-14
+5
-5
-1.5
+1.5
-6.5
+6.5
PK
PK
+1.5
-1.5
-3.5
+3.5
+10
-10
+2.5
-2.5
+10
-10
-8.5
+8.5
+5
-5
+13.5
-13.5
-3
+3
+9
-9
+5.5
-5.5
+2.5
-2.5
+2
-2
+8.5
-8.5
-1
+1
-1
+1
-5.5
+5.5
-6.5
+6.5
+3.5
-3.5
+13.5
-13.5
-6
+6
+12
-12
+12
-12
+10
-10
-2
+2
+8.5
-8.5
+1.5
-1.5
+9.5
-9.5
-4.5
+4.5
-5
+5
+3
-3
-1.5
+1.5
-1.5
+1.5
+12.5
-12.5
+10
-10
-7
+7
+5
-5
+7.5
-7.5
+3
-3
-3
+3
+3.5
-3.5
+3.5
-3.5
-7.5

+3
-3
-2
+2
+7
-7
+3
-3
+3
-3
+11.5
-11.5
+12
-12
+10
-10
-9
+9
+11.5
-11.5
-7.5
+7.5
-1.5
+1.5
-4.5
+4.5
+6
-6
-7
+7
-1
+1
-8.5
+8.5
-2
+2
-2
+2
+1
-1
+11.5
-11.5
+5
-5
+14
-14
-2
+2
+11
-11
+4
-4
+4.5
-4.5
+8
-8
+8.5
-8.5
-7
+7
-4
+4
-2
+2
-4
+4
+3.5
-3.5
+3
-3
+7.5
-7.5
+1.5
-1.5
+1
-1
+3.5
-3.5
-7
+7
+9
-9
+10
-10
+3.5
-3.5
+13
-13
-8
+8
-4.5
+4.5
+13
-13
+8.5
-8.5
+4
-4
-5
+5
-7
+7
+7.5
-7.5
-5.5
+5.5
+11
-11
+6
-6
-11.5
+11.5
-7
+7
-1.5
+1.5
+13.5
-13.5
-8
+8
+12
-12
+4
-4
+9.5
-9.5
+11
-11
+12
-12
+9.5
-9.5
+8
-8
+8.5
-8.5
+11.5
-11.5
-3.5
+3.5
+8
-8
-8
+8
-3
+3
+4
-4
+11
-11
+3
-3
+6.5
-6.5
-3.5
+3.5
+1
-1
-2
+2
-1.5
+1.5
+5
-5
-2.5
+2.5
-5.5
+5.5
+7.5
-7.5
+10.5
-10.5
+9
-9
+3
-3
+7
-7
-3.5
+3.5
-4
+4
+4
-4
+6.5
-6.5
PK
PK
+7.5
-7.5
+6.5
-6.5
+8
-8
+6
-6
-7.5
+7.5
-7.5
+7.5
-6.5
+6.5
+13
-13
+3.5
-3.5
+5
-5
-6.5
+6.5
-3
+3
+13
-13
+8.5
-8.5
+1.5
-1.5
-1.5
+1.5
+3.5
-3.5
-9
+9
+11.5
-11.5
+7.5
-7.5
+5.5
-5.5
+11
-11
+3.5
-3.5
+8
-8
+2.5
-2.5
-2
+2
-11
+11
+1
-1
+6.5
-6.5
+7
-7


-5
+9.5
-9.5
-3.5
+3.5
+2
-2
+5.5
-5.5
+11
-11
+8
-8
+6.5
-6.5
+1
-1
+7.5
-7.5
+6.5
-6.5
-1.5
+1.5
-2.5
+2.5
-1.5
+1.5
+4
-4
+12
-12
+2.5
-2.5
+6
-6
+11.5
-11.5
-6.5
+6.5
-1
+1
+5.5
-5.5
-4.5
+4.5
+8
-8
+7.5
-7.5
+14
-14
+5
-5
-2
+2
+2.5
-2.5
-1
+1
+10.5
-10.5
+4
-4
+8.5
-8.5
+11.5
-11.5
+10.5
-10.5
+1.5
-1.5
+7
-7
-2.5
+2.5
-5.5
+5.5
+4.5
-4.5
PK
PK
+10
-10
-7.5
+7.5
+3
-3
-7.5
+7.5
-5.5
+5.5
-2.5
+2.5
+2.5
-2.5
+11
-11
+2.5
-2.5
+7.5
-7.5
-2.5
+2.5
+10
-10
+6.5
-6.5
+3
-3
+14.5
-14.5
+2.5
-2.5
-2.5
+2.5
+6
-6
+4
-4
+5.5
-5.5
-1
+1
+4.5
-4.5
+12
-12
+6.5
-6.5
-4.5
+4.5
+7
-7
-2.5
+2.5
-8
+8
+4.5
-4.5
+5.5
-5.5
+10.5
-10.5
+10.5
-10.5
-1.5
+1.5
+6
-6
+5.5
-5.5
+5
-5
+1.5
-1.5
+2
-2
+7.5
-7.5
-6
+6
PK
PK
-6
+6
+2
-2
+1
-1
-3.5
+3.5
+6
-6
-5.5
+5.5
+4.5
-4.5
+9.5
-9.5
+9.5
-9.5
+7.5
-7.5
+2.5
-2.5
-6.5
+6.5
+8
-8
+3
-3
-1
+1
+1
-1
+3.5
-3.5
+9.5
-9.5
+4.5
-4.5
+1.5
-1.5
+4
-4
+3
-3
-5.5
+5.5
+6
-6
-13
+13
+5.5
-5.5
-4.5
+4.5
+12
-12
+1.5
-1.5
+10
-10
-6
+6
+1.5
-1.5
+15
-15
+1.5
-1.5
+3


-9
+9
+5
-5
+2
-2
+10
-10
+7
-7
+5
-5
+2.5
-2.5
-6
+6
+7.5
-7.5
+4
-4
-1.5
+1.5
+5
-5
-2
+2
+4
-4
+8
-8
-3
+3
-6
+6
+2.5
-2.5
+17
-17
+7
-7
+2.5
-2.5
+3.5
-3.5
+3.5
-3.5
-1.5
+1.5
+13.5
-13.5
+12.5
-12.5
+3
-3
+1.5
-1.5
+2
-2
-7
+7
-3.5
+3.5
-2
+2
+11.5
-11.5
-4.5
+4.5
-5
+5
+10
-10
+7
-7
+3.5
-3.5
+3
-3
+13.5
-13.5
-11
+11
-5
+5
+8
-8
-1.5
+1.5
+7
-7
+11
-11
-4
+4
-3.5
+3.5
+10.5
-10.5
+12.5
-12.5
+1.5
-1.5
-2.5
+2.5
-4
+4
+15
-15
+5.5
-5.5
+7
-7
+6.5
-6.5
-6.5
+6.5
+17.5
-17.5
+4
-4
-2
+2
+15.5
-15.5
+1
-1
+7
-7
+9.5
-9.5
-2.5
+2.5
+4.5
-4.5
-1
+1
+7
-7
-6.5
+6.5
+6
-6
-5.5
+5.5
+10.5
-10.5
-11
+11
-1
+1
+2
-2
+6
-6
+11.5
-11.5
+10.5
-10.5
+9
-9
+9.5
-9.5
+4.5
-4.5
+10.5
-10.5
-1
+1
+2.5
-2.5
+8
-8
-6.5
+6.5
+5
-5
+5.5
-5.5
+5
-5
-7
+7
+4
-4
+9
-9
-2
+2
+6.5
-6.5
+11
-11
+10
-10
-11
+11
+1.5
-1.5
+11
-11
+2
-2
+4.5
-4.5
-3
+3
+4.5
-4.5
+2.5
-2.5
+8.5
-8.5
+7
-7
+2.5
-2.5
+13.5
-13.5
+1.5
-1.5
+2
-2
+3.5
-3.5
+8.5
-8.5
+1.5
-1.5
+11
-11
-7.5
+7.5
+12
-12
+5
-5
+11
-11
+11
-11
+11
-11


In [50]:
spreaddb['Spread'] = spreaddb['Spread'].replace('PK',0)
spreaddb['Spread'] = spreaddb['Spread'].apply(lambda x: float(x))

In [51]:
#spreaddb['Date'].apply(lambda x: int(x))
#moneydb['Date'].apply(lambda x: int(x))
moneydb['ML Odds'] = moneydb['ML Odds'].apply(lambda x: int(x))

In [52]:
for i, date in enumerate(db['Date']):
    
    db['Odds'][i] = moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds']
    print("Team:", db['Team'][i], "Odds:", moneydb[(moneydb['Date'] == date) & (moneydb['Team'] == db['Team'][i])]['ML Odds'])
    db['Line'][i] = spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])]['Spread']
    print("Team:", db['Team'][i], "Spread:", spreaddb[(spreaddb['Date'] == date) & (spreaddb['Team'] == db['Team'][i])]['Spread'])

C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\helgeson\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


Team: CLE Odds: 10    600
Name: ML Odds, dtype: int64
Team: CLE Spread: 10    11.5
Name: Spread, dtype: float64
Team: CLE Odds: 30    305
Name: ML Odds, dtype: int64
Team: CLE Spread: 30    8.5
Name: Spread, dtype: float64
Team: CLE Odds: 41   -260
Name: ML Odds, dtype: int64
Team: CLE Spread: 41   -6.5
Name: Spread, dtype: float64
Team: CLE Odds: 49   -150
Name: ML Odds, dtype: int64
Team: CLE Spread: 49   -3.0
Name: Spread, dtype: float64
Team: CLE Odds: 70    250
Name: ML Odds, dtype: int64
Team: CLE Spread: 70    7.0
Name: Spread, dtype: float64
Team: CLE Odds: 87    245
Name: ML Odds, dtype: int64
Team: CLE Spread: 87    6.0
Name: Spread, dtype: float64
Team: CLE Odds: 97   -170
Name: ML Odds, dtype: int64
Team: CLE Spread: 97   -4.0
Name: Spread, dtype: float64
Team: CLE Odds: 117    195
Name: ML Odds, dtype: int64
Team: CLE Spread: 117    6.0
Name: Spread, dtype: float64
Team: CLE Odds: 128    380
Name: ML Odds, dtype: int64
Team: CLE Spread: 128    9.5
Name: Spread, dtype: floa

Team: CLE Spread: 1305    6.0
Name: Spread, dtype: float64
Team: CLE Odds: 1338    700
Name: ML Odds, dtype: int64
Team: CLE Spread: 1338    13.0
Name: Spread, dtype: float64
Team: CLE Odds: 1340    415
Name: ML Odds, dtype: int64
Team: CLE Spread: 1340    10.0
Name: Spread, dtype: float64
Team: CLE Odds: 1374    125
Name: ML Odds, dtype: int64
Team: CLE Spread: 1374    2.5
Name: Spread, dtype: float64
Team: CLE Odds: 1378    400
Name: ML Odds, dtype: int64
Team: CLE Spread: 1378    10.0
Name: Spread, dtype: float64
Team: CLE Odds: 1402    1200
Name: ML Odds, dtype: int64
Team: CLE Spread: 1402    16.0
Name: Spread, dtype: float64
Team: CLE Odds: 1411    325
Name: ML Odds, dtype: int64
Team: CLE Spread: 1411    8.5
Name: Spread, dtype: float64
Team: CLE Odds: 1439    230
Name: ML Odds, dtype: int64
Team: CLE Spread: 1439    7.0
Name: Spread, dtype: float64
Team: BOS Odds: 1459   -210
Name: ML Odds, dtype: int64
Team: BOS Spread: 1459   -5.0
Name: Spread, dtype: float64
Team: BOS Odds: 

Team: BOS Spread: 1802    8.0
Name: Spread, dtype: float64
Team: BOS Odds: 1162   -110
Name: ML Odds, dtype: int64
Team: BOS Spread: 1162   -1.0
Name: Spread, dtype: float64
Team: BOS Odds: 1810   -205
Name: ML Odds, dtype: int64
Team: BOS Spread: 1810   -5.0
Name: Spread, dtype: float64
Team: BOS Odds: 1194   -123
Name: ML Odds, dtype: int64
Team: BOS Spread: 1194   -1.5
Name: Spread, dtype: float64
Team: BOS Odds: 1215   -335
Name: ML Odds, dtype: int64
Team: BOS Spread: 1215   -8.0
Name: Spread, dtype: float64
Team: BOS Odds: 1223   -525
Name: ML Odds, dtype: int64
Team: BOS Spread: 1223   -10.0
Name: Spread, dtype: float64
Team: BOS Odds: 1243   -155
Name: ML Odds, dtype: int64
Team: BOS Spread: 1243   -3.0
Name: Spread, dtype: float64
Team: BOS Odds: 1260    130
Name: ML Odds, dtype: int64
Team: BOS Spread: 1260    2.0
Name: Spread, dtype: float64
Team: BOS Odds: 1816   -165
Name: ML Odds, dtype: int64
Team: BOS Spread: 1816   -3.5
Name: Spread, dtype: float64
Team: BOS Odds: 1299

Team: WAS Odds: 994   -125
Name: ML Odds, dtype: int64
Team: WAS Spread: 994   -3.0
Name: Spread, dtype: float64
Team: WAS Odds: 1002    145
Name: ML Odds, dtype: int64
Team: WAS Spread: 1002    3.5
Name: Spread, dtype: float64
Team: WAS Odds: 1026    450
Name: ML Odds, dtype: int64
Team: WAS Spread: 1026    10.5
Name: Spread, dtype: float64
Team: WAS Odds: 2044    170
Name: ML Odds, dtype: int64
Team: WAS Spread: 2044    4.0
Name: Spread, dtype: float64
Team: WAS Odds: 1059   -110
Name: ML Odds, dtype: int64
Team: WAS Spread: 1059    0.0
Name: Spread, dtype: float64
Team: WAS Odds: 1760    170
Name: ML Odds, dtype: int64
Team: WAS Spread: 1760    4.5
Name: Spread, dtype: float64
Team: WAS Odds: 1784    335
Name: ML Odds, dtype: int64
Team: WAS Spread: 1784    9.5
Name: Spread, dtype: float64
Team: WAS Odds: 1143   -123
Name: ML Odds, dtype: int64
Team: WAS Spread: 1143   -1.5
Name: Spread, dtype: float64
Team: WAS Odds: 1149   -225
Name: ML Odds, dtype: int64
Team: WAS Spread: 1149   

...

Team: HOU Odds: 6576   -250
Name: ML Odds, dtype: int64
Team: HOU Spread: 6576   -6.0
Name: Spread, dtype: float64
Team: HOU Odds: 6593   -230
Name: ML Odds, dtype: int64
Team: HOU Spread: 6593   -5.0
Name: Spread, dtype: float64
Team: HOU Odds: 5453   -550
Name: ML Odds, dtype: int64
Team: HOU Spread: 5453   -9.5
Name: Spread, dtype: float64
Team: HOU Odds: 5467   -480
Name: ML Odds, dtype: int64
Team: HOU Spread: 5467   -9.0
Name: Spread, dtype: float64
Team: HOU Odds: 7171   -420
Name: ML Odds, dtype: int64
Team: HOU Spread: 7171   -8.5
Name: Spread, dtype: float64
Team: HOU Odds: 5490   -240
Name: ML Odds, dtype: int64
Team: HOU Spread: 5490   -5.5
Name: Spread, dtype: float64
Team: HOU Odds: 5510   -125
Name: ML Odds, dtype: int64
Team: HOU Spread: 5510   -2.5
Name: Spread, dtype: float64
Team: HOU Odds: 5529   -550
Name: ML Odds, dtype: int64
Team: HOU Spread: 5529   -9.5
Name: Spread, dtype: float64
Team: HOU Odds: 5546   -150
Name: ML Odds, dtype: int64
Team: HOU Spread: 5546  

Team: OKC Spread: 5212    1.0
Name: Spread, dtype: float64
Team: OKC Odds: 5263   -103
Name: ML Odds, dtype: int64
Team: OKC Spread: 5263    0.0
Name: Spread, dtype: float64
Team: OKC Odds: 6521   -170
Name: ML Odds, dtype: int64
Team: OKC Spread: 6521   -4.0
Name: Spread, dtype: float64
Team: OKC Odds: 5300   -117
Name: ML Odds, dtype: int64
Team: OKC Spread: 5300   -1.0
Name: Spread, dtype: float64
Team: OKC Odds: 5318    200
Name: ML Odds, dtype: int64
Team: OKC Spread: 5318    5.5
Name: Spread, dtype: float64
Team: OKC Odds: 5323   -290
Name: ML Odds, dtype: int64
Team: OKC Spread: 5323   -8.0
Name: Spread, dtype: float64
Team: OKC Odds: 7039   -145
Name: ML Odds, dtype: int64
Team: OKC Spread: 7039   -2.5
Name: Spread, dtype: float64
Team: OKC Odds: 5366   -135
Name: ML Odds, dtype: int64
Team: OKC Spread: 5366   -2.0
Name: Spread, dtype: float64
Team: OKC Odds: 5382    155
Name: ML Odds, dtype: int64
Team: OKC Spread: 5382    4.0
Name: Spread, dtype: float64
Team: OKC Odds: 5409 

Team: TOR Spread: 7368   -3.5
Name: Spread, dtype: float64
Team: TOR Odds: 6464    335
Name: ML Odds, dtype: int64
Team: TOR Spread: 6464    8.0
Name: Spread, dtype: float64
Team: TOR Odds: 5110    135
Name: ML Odds, dtype: int64
Team: TOR Spread: 5110    3.0
Name: Spread, dtype: float64
Team: TOR Odds: 5136   -150
Name: ML Odds, dtype: int64
Team: TOR Spread: 5136   -3.0
Name: Spread, dtype: float64
Team: TOR Odds: 6469   -2000
Name: ML Odds, dtype: int64
Team: TOR Spread: 6469   -14.5
Name: Spread, dtype: float64
Team: TOR Odds: 6481   -800
Name: ML Odds, dtype: int64
Team: TOR Spread: 6481   -11.5
Name: Spread, dtype: float64
Team: TOR Odds: 5195   -600
Name: ML Odds, dtype: int64
Team: TOR Spread: 5195   -10.5
Name: Spread, dtype: float64
Team: TOR Odds: 6505   -420
Name: ML Odds, dtype: int64
Team: TOR Spread: 6505   -9.0
Name: Spread, dtype: float64
Team: TOR Odds: 5215    103
Name: ML Odds, dtype: int64
Team: TOR Spread: 5215   -1.5
Name: Spread, dtype: float64
Team: TOR Odds: 7

Team: ORL Odds: 4948    700
Name: ML Odds, dtype: int64
Team: ORL Spread: 4948    12.0
Name: Spread, dtype: float64
Team: ORL Odds: 4966   -220
Name: ML Odds, dtype: int64
Team: ORL Spread: 4966   -5.0
Name: Spread, dtype: float64
Team: ORL Odds: 4981   -117
Name: ML Odds, dtype: int64
Team: ORL Spread: 4981   -1.0
Name: Spread, dtype: float64
Team: ORL Odds: 4997   -220
Name: ML Odds, dtype: int64
Team: ORL Spread: 4997   -5.0
Name: Spread, dtype: float64
Team: ORL Odds: 6972    245
Name: ML Odds, dtype: int64
Team: ORL Spread: 6972    7.0
Name: Spread, dtype: float64
Team: ORL Odds: 6371   -150
Name: ML Odds, dtype: int64
Team: ORL Spread: 6371   -2.0
Name: Spread, dtype: float64
Team: ORL Odds: 5021    160
Name: ML Odds, dtype: int64
Team: ORL Spread: 5021    4.0
Name: Spread, dtype: float64
Team: ORL Odds: 5038    380
Name: ML Odds, dtype: int64
Team: ORL Spread: 5038    9.0
Name: Spread, dtype: float64
Team: ORL Odds: 6414    255
Name: ML Odds, dtype: int64
Team: ORL Spread: 6414 

Team: ORL Spread: 7304    3.5
Name: Spread, dtype: float64
Team: ORL Odds: 6196    435
Name: ML Odds, dtype: int64
Team: ORL Spread: 6196    11.0
Name: Spread, dtype: float64
Team: ORL Odds: 6905   -145
Name: ML Odds, dtype: int64
Team: ORL Spread: 6905   -2.5
Name: Spread, dtype: float64
Team: ORL Odds: 6921    155
Name: ML Odds, dtype: int64
Team: ORL Spread: 6921    4.0
Name: Spread, dtype: float64
Team: ORL Odds: 6272    325
Name: ML Odds, dtype: int64
Team: ORL Spread: 6272    7.5
Name: Spread, dtype: float64
Team: ORL Odds: 6295   -135
Name: ML Odds, dtype: int64
Team: ORL Spread: 6295   -2.0
Name: Spread, dtype: float64


In [53]:
spreaddb.to_csv('Spreads.csv')

In [54]:
moneydb.to_csv('MLodds.csv')

In [56]:
db.to_csv('2017_2019stats.csv')